# Machine Learning

We will use all the prior order information to generate features, and use the train data to create the target variables.

From exploratory analysis, we learned that purchase pattern at the department level is pretty consistence, and the reorder pattern are similar to orders created by users before. Also, reorder on average fall under 15 days for new clients, and gradually decrease over time to 2-4 days. Hence, we will test the following features
- product purchase frequency
- product purchase recency

### Import modules and data

In [33]:
import pandas as pd
import gc
from pycaret.classification import *
from sklearn.model_selection import RandomizedSearchCV

In [34]:
# aisles = pd.read_csv('./data/aisles.csv')
# dept = pd.read_csv('./data/departments.csv')
orders = pd.read_csv('./data/orders.csv')
# products = pd.read_csv('./data/products.csv')
orders_p = pd.read_csv('./data/order_products__prior.csv')
orders_tr = pd.read_csv('./data/order_products__train.csv')

In [35]:
prior_order = orders.query('eval_set == "prior"')
train_order = orders.query('eval_set == "train"')
test_order = orders.query('eval_set == "test"')

In [36]:
print(orders.shape)
display(orders.head())
print(orders_p.shape)
print(orders_tr.shape)
display(orders_p.head(), orders_tr.head())

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


(32434489, 4)
(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


### Feature Creations

- last purchase: whether a particular product was purchased in the latest order by user
- last 3 purchase: product appear in the last 3 purchases
- the duration (in terms of days) that an item was purchased since last purchase (will be zero if the product didn't exist in latest orders)
- how many times an item has been purchased by the user
- how often that item appear in user purchase history, in form of percentage

##### Obtain a list of last purchase order by user

In [37]:
last_purchase = prior_order[prior_order['order_number'] == prior_order.groupby(
    ['user_id'])['order_number'].transform('max')]
last_purchase_list = last_purchase['order_id'].tolist()

##### Generating last purchase feature

In [38]:
order_detail = orders_p.merge(prior_order[['order_id', 'user_id', 'order_number']],
                              on=['order_id'], how='left')
order_detail.loc[order_detail['order_id'].isin(last_purchase_list),
                 'last_purchase'] = 1
order_detail.loc[~order_detail['order_id'].isin(last_purchase_list),
                 'last_purchase'] = 0

##### Generating times of purchases and product appear frequency

In [39]:
user_df = order_detail.groupby(['user_id', 'product_id']).agg({
    'order_number': 'max', 'last_purchase': 'max', 'reordered': 'sum'})

In [40]:
user_df['product_appear'] = (user_df['reordered']+1)/user_df['order_number']

In [41]:
user_order_cnt = orders.groupby(['user_id', 'order_number']).agg({
    'days_since_prior_order': 'sum'})

##### Generate consecutive purchase features for the last 3 orders, by user and by item

In [42]:
# Get the last 3 purchases
user_order_cnt_1 = user_order_cnt.groupby(level=0).apply(lambda df: df[-4:])
user_order_cnt_1.index = user_order_cnt_1.index.droplevel(0)

In [43]:
last_three = user_order_cnt_1.reset_index().drop(
    'days_since_prior_order', axis=1)

In [44]:
product_last_three = pd.merge(last_three, order_detail,
                              on=['user_id', 'order_number'], how='inner')
product_cnt = product_last_three.groupby(['user_id', 'product_id']).agg(
    {'product_id': 'count'})

##### Forming final training dataframe

In [46]:
temp_df = pd.merge(user_df, product_cnt, left_index=True,
                   right_index=True, how='left')

In [47]:
temp_df = temp_df.rename(columns={'product_id': 'buy_cnt'})

In [48]:
temp_df.reset_index(inplace=True)

In [49]:
# getting user id within the train dataset section
train_id = train_order.user_id.unique().tolist()

In [50]:
# filter out the training set user id from the temp dataframe
temp_train_df = temp_df.loc[temp_df['user_id'].isin(train_id)]

Generating target features from the train data

In [51]:
train_target = pd.merge(train_order[['order_id', 'user_id']], orders_tr,
                        on=['order_id'], how='left')
train_train = train_target.drop(['order_id', 'add_to_cart_order'], axis=1)
train_target = train_target.rename(columns={'reordered': 'target'})

# set to 1 as these are the products brought in the next purchase
train_target['target'] = 1

Add target feature to the training dataframe, and drop order_number column

In [52]:
train_df = pd.merge(temp_train_df, train_target, on=['user_id', 'product_id'],
                    how='outer').fillna(0)
train_df = train_df.set_index(['user_id', 'product_id'])

# columns that won't be needed for final features
train_df.drop(['order_number', 'order_id', 'add_to_cart_order'],
              axis=1, inplace=True)

Set target to integer to ensure model recognize that as binary variable

In [53]:
train_df = train_df.astype({'target': 'int'})

In [54]:
train_df.head()

last_purchase  reordered  product_appear  buy_cnt  target
user_id product_id                                                           
1       196                   1.0        9.0             1.0      3.0       1
        10258                 1.0        8.0             0.9      3.0       1
        10326                 0.0        0.0             0.2      0.0       0
        12427                 1.0        9.0             1.0      3.0       0
        13032                 1.0        2.0             0.3      1.0       1

##### Create separate formatted test dataset for ease of access later.

In [55]:
test_id = test_order.user_id.unique().tolist()
df_for_submit = temp_df.loc[temp_df['user_id'].isin(test_id)].set_index(
    ['user_id', 'product_id'])

In [56]:
df_for_submit.to_csv('./data/df_submit.csv')

### Modeling

##### Setup

In [57]:
gc.collect()
exp = setup(train_df, target='target', categorical_features=['last_purchase'],
            train_size=.8)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,3459
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(9030454, 5)"
4,Missing Values,False
5,Numeric Features,3
6,Categorical Features,1
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


##### Comparing model to see if any models standout from the F1 score perspective

In [26]:
compare_models(blacklist=['knn', 'ridge', 'svm', 'lda', 'nb', 'qda', 'et', 'catboost'],
               fold=2, round=3, sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Extreme Gradient Boosting,0.914000,0.882000,0.501000,0.889000,0.641000,0.596000
1,Light Gradient Boosting Machine,0.914000,0.882000,0.501000,0.890000,0.641000,0.597000
2,Ada Boost Classifier,0.913000,0.881000,0.501000,0.884000,0.640000,0.595000
3,Gradient Boosting Classifier,0.914000,0.882000,0.498000,0.893000,0.640000,0.595000
4,Random Forest Classifier,0.912000,0.876000,0.497000,0.879000,0.635000,0.590000
5,Decision Tree Classifier,0.913000,0.872000,0.493000,0.888000,0.634000,0.589000
6,Logistic Regression,0.848000,0.705000,0.035000,0.588000,0.066000,0.050000


**Observation**: It turns major classification models have similar F1 score performance except Logistic Regression. Most models are able to have 0.5 "Recall", which mean out of all the true qualified reordered items, half of them are identified. Percision is at 0.89, which is pretty decent given no tuning has been made yet.
<br>
There could be room to increase the recall rate in order to maximize the F1 score (objective of the problem), assuming the trade off from precision will worth it. We will test will lower the binary cut off rate to 0.4 so see if improvement exist.

Base on the result above, we will test Extreme Gradient Boosting, Light Gradient Boosting Machine. Also added Random Forest to get range of performance when apply different probability thresholds.
<br>

To save time, we will create the function below to automate the steps.

In [27]:

def model_f1(model, prob):
    """ Automate the step on generating models and prediction

    Args:
        model: the abbreviated string for estimators
        prob: probabiliy therhold to determin whether an output is 0 or 1

    Returns:
        name: created model
        var: prediction generated from the model
    """
    name = create_model(model, fold=2)
    var = predict_model(name, probability_threshold=prob)
    return name, var

In [28]:
%%time
lightgbm, lg_pred = model_f1('lightgbm', .4)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Light Gradient Boosting Machine,0.9127,0.8826,0.5409,0.8307,0.6552,0.6078


Wall time: 55.9 s


In [29]:
%%time
xgb, xgb_pred = model_f1('xgboost', .4)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Extreme Gradient Boosting,0.9121,0.8824,0.5461,0.8205,0.6558,0.6077


Wall time: 1min 27s


In [30]:
%%time
rfc, rfc_pred = model_f1('rf', .4)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Random Forest Classifier,0.9116,0.8795,0.5383,0.8246,0.6514,0.6033


Wall time: 1min 42s


**Obseravtion**: The trade off appear to be equal among the increase in Recall but decrease in Precision. The net gain in F1 score is 0.01, not significant enough to justify the adjustment, at least from the training set data. However, the test set data are much smaller and in that case it may worth lower the probability in order minimize penalty from low recall.

#### Save all the models for final test data submission

In [30]:
path = './data/save_model/'

def final_save(model, name):
    """ create the final model and save that as pkl file

    Args:
    model: model created with the sample data

    Returns:
    finalize model using all the data
    """

    final = finalize_model(model)
    save_model(final, model_name=path+name)

In [32]:
final_save(rfc, 'rfc')
final_save(lightgbm, 'lgbm')
final_save(xgb, 'xgb')

Transformation Pipeline and Model Succesfully Saved


#### Blend Models

Test out the the output by blending models, performance is inferior than individual model, hence won't proceed with that strategy

In [33]:
%%time
blend = blend_models(estimator_list=[rfc, xgb, lightgbm], fold=2)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.9138,0.0,0.4994,0.8902,0.6399,0.5953
1,0.9141,0.0,0.5004,0.8922,0.6412,0.5967
Mean,0.9140,0.0,0.4999,0.8912,0.6405,0.5960
SD,0.0002,0.0,0.0005,0.0010,0.0006,0.0007


Wall time: 3min 51s


**Observation**: No major improvement on F1 score by blending models, won't pursue this strategy.

### Model Tuning

After initial data submission, xgb has the highest F1 score on kaggle (0.359), with random forecast being a close second (0.357). We will tune the model to see if further improvement exist.

In [26]:
rfc_tune = tune_model('rf', fold=2)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.9144,0.8831,0.4994,0.8965,0.6415,0.5973
1,0.9142,0.8827,0.4992,0.8949,0.6409,0.5966
Mean,0.9143,0.8829,0.4993,0.8957,0.6412,0.5969
SD,0.0001,0.0002,0.0001,0.0008,0.0003,0.0004


In [31]:
final_save(rfc_tune, 'rfc_tune')

Transformation Pipeline and Model Succesfully Saved


**Note on execution**: xgboost tuning will take a long time. As we already know the base model performance from above. We re-executed the setup step to take only **1%** of sample in order to get the tuned parameters

In [58]:
xgb_tune = tune_model('xgboost', fold=2)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.9079,0.8667,0.5088,0.8228,0.6287,0.5795
1,0.9072,0.8641,0.4940,0.8326,0.6201,0.5712
Mean,0.9075,0.8654,0.5014,0.8277,0.6244,0.5753
SD,0.0003,0.0013,0.0074,0.0049,0.0043,0.0042


In [59]:
final_save(xgb_tune, 'xgb_tune')

Transformation Pipeline and Model Succesfully Saved
